# Description

This notebooks will show the structure of the main data matrices in PhenoPLIER, and will guide you in analyzing gene associations for a particular trait: neutrophil counts, which is presented in the [manuscript](https://greenelab.github.io/phenoplier_manuscript/#phenoplier-an-integration-framework-based-on-gene-co-expression-patterns) in Figure 1c.

# Modules

In [1]:
import tempfile

import numpy as np
from scipy import stats
import pandas as pd

from entity import Trait, Gene
import conf

# Load gene module-gene membership matrix (matrix Z)

Here we load the gene module-gene membership matri, or "latent variables loadings matrix" (from the terminology of the [MultiPLIER article](https://doi.org/10.1016/j.cels.2019.04.003)).

In [2]:
matrix_z = pd.read_pickle(conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"])

In [3]:
matrix_z.shape

(6750, 987)

In [4]:
matrix_z.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


As you can see, this matrix Z contains the membership value for each gene across all LVs (or gene modules).
A value of zero means that the gene does not belong to that LV, whereas a larger value represents how strongly that gene belongs to the LV.
A group of genes that belong to the same LV represent a gene-set that has a similar expression profile across a set of tissues or cell types.
We'll cover this in more detail in the next notebook (`02-LV_cell_types-...`).

# Load information about LV alignment with pathways

LV in matrix Z can represent a group of genes that align well with prior pathways (or prior knowledge) or be "novel" in the sense that the combination of genes do not represent a known unit but was found the PLIER when factorizing the recount2 data (see the MultiPLIER article for more details).

Here we load that information, where for each LV and pathway, we have a p-value and area under the curve (AUC) that indicate how well the LV aligns to that pathway.

In [5]:
lv_metadata = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [6]:
lv_metadata.shape

(2157, 5)

In [7]:
lv_metadata.head()

,pathway,LV index,AUC,p-value,FDR
1,KEGG_LYSINE_DEGRADATION,1,0.388059,0.866078,0.956005
2,REACTOME_MRNA_SPLICING,1,0.733057,0.000048,0.000582
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,0.001628,0.011366
4,KEGG_DNA_REPLICATION,1,0.549473,0.312155,0.539951
5,PID_MYC_ACTIVPATHWAY,1,0.639303,0.021702,0.083739


# Load gene associations from PhenomeXcan

Now we load the gene association across ~4,000 traits from [PhenomeXcan](https://doi.org/10.1126/sciadv.aba2083).
The file we load here are the Summary-MultiXcan (or S-MultiXcan) results, essentially a p-value for each gene-trait pair.
In the notebook I refer to these results generically as "TWAS results", meaning that we have gene-trait associations.
All these TWAS results were derived solely from GWAS summary stats, so you can also generate yours relatively easily by using [S-MultiXcan](https://doi.org/10.1371/journal.pgen.1007889).

In [8]:
phenomexcan_df = pd.read_pickle(conf.PHENOMEXCAN["SMULTIXCAN_MASHR_PVALUES_FILE"])

In [9]:
phenomexcan_df.shape

(22515, 4091)

In [10]:
phenomexcan_df.head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.865429,0.918314,0.810683,0.374671,0.189032,0.140981,0.467741,0.129427,0.193680,0.285479,...,0.416210,0.782554,0.609467,0.980281,0.666504,0.409761,0.713310,0.168319,0.460244,0.765506
ENSG00000000457,0.174192,0.064765,0.889194,0.896938,0.448596,0.269602,0.540261,0.068405,0.041813,0.313427,...,0.149360,0.512603,0.010907,0.228982,0.607081,0.812484,0.678749,0.918971,0.311187,0.344574
ENSG00000000460,0.879969,0.240715,0.238228,0.567555,0.921320,0.825036,0.782230,0.644525,0.392273,0.840014,...,0.503520,0.764147,0.587969,0.301460,0.629621,0.486664,0.736509,0.933600,0.000477,0.321223
ENSG00000000938,0.192670,0.400054,0.114353,0.470700,0.889202,0.000011,0.899764,0.212352,0.829671,0.372348,...,0.899212,0.961678,0.059247,0.588855,0.898525,0.135045,0.954998,0.088220,0.176497,0.304281
ENSG00000000971,0.180632,0.793060,0.490585,0.088752,0.744531,0.949639,0.253817,0.377408,0.971655,0.070266,...,0.390618,0.093824,0.020391,0.109883,0.870551,0.995450,0.002660,0.421588,0.656851,0.868416


Convert gene Ensembl IDs to symbols:

In [11]:
phenomexcan_df = phenomexcan_df.rename(index=Gene.GENE_ID_TO_NAME_MAP)

In [12]:
phenomexcan_df = phenomexcan_df.loc[~phenomexcan_df.index.duplicated()]

In [13]:
phenomexcan_df.head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
gene_name,,,,,,,,,,,,,,,,,,,,,
DPM1,0.865429,0.918314,0.810683,0.374671,0.189032,0.140981,0.467741,0.129427,0.193680,0.285479,...,0.416210,0.782554,0.609467,0.980281,0.666504,0.409761,0.713310,0.168319,0.460244,0.765506
SCYL3,0.174192,0.064765,0.889194,0.896938,0.448596,0.269602,0.540261,0.068405,0.041813,0.313427,...,0.149360,0.512603,0.010907,0.228982,0.607081,0.812484,0.678749,0.918971,0.311187,0.344574
C1orf112,0.879969,0.240715,0.238228,0.567555,0.921320,0.825036,0.782230,0.644525,0.392273,0.840014,...,0.503520,0.764147,0.587969,0.301460,0.629621,0.486664,0.736509,0.933600,0.000477,0.321223
FGR,0.192670,0.400054,0.114353,0.470700,0.889202,0.000011,0.899764,0.212352,0.829671,0.372348,...,0.899212,0.961678,0.059247,0.588855,0.898525,0.135045,0.954998,0.088220,0.176497,0.304281
CFH,0.180632,0.793060,0.490585,0.088752,0.744531,0.949639,0.253817,0.377408,0.971655,0.070266,...,0.390618,0.093824,0.020391,0.109883,0.870551,0.995450,0.002660,0.421588,0.656851,0.868416


Keep genes present in our matrix Z only:

In [14]:
common_genes = phenomexcan_df.index.intersection(matrix_z.index)
display(common_genes)

Index(['DPM1', 'FGR', 'CFH', 'GCLC', 'NFYA', 'CYP51A1', 'RAD52', 'BAD', 'LAP3',
       'HS3ST1',
       ...
       'NEFL', 'CCL3', 'PSMB3', 'SSTR3', 'DHRS11', 'ACACA', 'HIST1H3H',
       'MRPL45', 'LINC00921', 'ADORA3'],
      dtype='object', length=6452)

In [15]:
phenomexcan_df = phenomexcan_df.loc[common_genes]

In [16]:
phenomexcan_df.shape

(6452, 4091)

In [17]:
matrix_z = matrix_z.loc[common_genes]

In [18]:
matrix_z.shape

(6452, 987)

# Take a look at genes associated with neutrophil counts

Below I search the PhenomeXcan results to find traits related to "neutrophils".

In [19]:
phenomexcan_df.columns[phenomexcan_df.columns.str.lower().str.contains("neutrophil")]

Index(['30140_raw-Neutrophill_count', '30200_raw-Neutrophill_percentage',
       'Astle_et_al_2016_Sum_neutrophil_eosinophil_counts',
       'Astle_et_al_2016_Neutrophil_count',
       'Astle_et_al_2016_Sum_basophil_neutrophil_counts'],
      dtype='object')

For this demo, I select the the "neutrophil count" TWAS derived from the GWAS performed by [Astle et. al](https://doi.org/10.1016/j.cell.2016.10.042). Below you can see the sample size:

In [20]:
trait_code = "Astle_et_al_2016_Neutrophil_count"
t = Trait.get_trait(full_code=trait_code)
display(f"{trait_code} - sample size: {t.n}")

'Astle_et_al_2016_Neutrophil_count - sample size: 173480'

In [21]:
traits_df = phenomexcan_df[[trait_code]].dropna()

In [22]:
traits_df.shape

(6452, 1)

In [23]:
traits_df.head()

,Astle_et_al_2016_Neutrophil_count
DPM1,0.388682
FGR,0.886969
CFH,0.442834
GCLC,0.569352
NFYA,0.001760


Here I quickly show the data summary for this trait's gene associations:

In [24]:
traits_df.apply(lambda x: -np.log10(x)).describe()

,Astle_et_al_2016_Neutrophil_count
count,6452.000000
mean,1.448715
std,6.349477
min,0.000051
25%,0.214177
50%,0.560065
75%,1.227759
max,260.901281


Make sure we don't have missing values or NaN.

In [32]:
assert not traits_df.isna().any().any()
assert not np.isinf(traits_df).any().any()

# Convert p-values to z-scores

This converts a p-value to a scalar > 0, where higher positive values mean stronger association, and lower values close to zero mean weaker associations. Check our manuscript for more details on this.

In [33]:
traits_zscores = pd.DataFrame(
    data=np.abs(stats.norm.ppf(traits_df / 2)),
    index=traits_df.index.copy(),
    columns=traits_df.columns.copy(),
)

In [35]:
assert not traits_zscores.isna().any().any()
assert not np.isinf(traits_zscores).any().any()

In [36]:
traits_zscores.head()

,Astle_et_al_2016_Neutrophil_count
DPM1,0.862010
FGR,0.142140
CFH,0.767416
GCLC,0.569006
NFYA,3.127917


In [37]:
traits_zscores.describe()

,Astle_et_al_2016_Neutrophil_count
count,6452.000000
mean,1.480574
std,1.723161
min,0.000146
25%,0.509084
50%,1.090753
75%,1.886786
max,34.553676


# Analysis of a neutrophil-termed LV

Let's take as an example an LV that was previously analyzed in the MultiPLIER study, which we identify as `LV603`. This LV aligns well with pathways related to neutrophils, as you can see below.

In [38]:
lv_metadata[
    (lv_metadata["LV index"] == "603") & (lv_metadata["FDR"] < 0.05)
].sort_values("FDR")

,pathway,LV index,AUC,p-value,FDR
1511,IRIS_Neutrophil-Resting,603,0.905751,8.355936e-38,4.505939e-35
1512,SVM Neutrophils,603,0.979789,2.856571e-11,1.432936e-09
1513,PID_IL8CXCR2_PATHWAY,603,0.810732,8.814671e-04,7.041943e-03
1516,SIG_PIP3_SIGNALING_IN_B_LYMPHOCYTES,603,0.769292,3.387907e-03,1.948724e-02


Let's see which genes more strongly belong to LV603 (the numbers are the gene weights in this LV):

In [39]:
lv603_top_genes = matrix_z["LV603"].sort_values(ascending=False)
display(lv603_top_genes.head(20))

CXCR2        5.320459
FCGR3B       5.128372
TNFRSF10C    5.035457
VNN2         4.680865
ZDHHC18      4.495976
MNDA         4.488505
CXCR1        4.442062
P2RY13       4.404405
VNN3         4.253184
FPR2         4.187560
CEACAM3      4.139476
C5AR1        4.101986
SLC45A4      4.068913
AQP9         3.939923
CCR3         3.883533
ABTB1        3.745259
CSF3R        3.735651
FPR1         3.720018
DPEP2        3.656125
SIRPB1       3.632251
Name: LV603, dtype: float64

Are these top genes associated with our neutrophil count?

In [40]:
traits_zscores.loc[lv603_top_genes.index].head(20)

,Astle_et_al_2016_Neutrophil_count
CXCR2,4.853946
FCGR3B,2.021852
TNFRSF10C,5.384578
VNN2,2.597925
ZDHHC18,7.438307
MNDA,0.667815
CXCR1,6.610157
P2RY13,3.057785
VNN3,1.273416
FPR2,0.271665


It seems so. But what about the rest of the genes? They might be also strongly associated.
Let's take a random sample:

In [41]:
traits_zscores.sample(n=20, random_state=0)

,Astle_et_al_2016_Neutrophil_count
CPSF2,0.162255
SLC7A7,1.150304
GGT1,1.545870
PIK3C3,0.818348
CHIT1,1.434727
ZNF570,0.176574
MRPL9,1.672936
COPE,0.860495
KLRK1,2.009665
DVL3,1.381302


They do not seem as high as those within the top genes in LV603.
If we compute the correlation between LV603 gene weights (`lv603_top_genes`) and gene associations for neutrophil counts (`traits_zscores`) we get this:

In [42]:
stats.pearsonr(
    traits_zscores.loc[lv603_top_genes.index].iloc[:, 0].to_numpy(),
    lv603_top_genes.to_numpy(),
)

(0.07636590129740203, 8.158019888806208e-10)

Although the correlation is significant (`8.16e-10`) and the slope positive (we are interested only in genes at the top of the LV), we need to account for correlated predicted expression from the TWAS models (for example, if the expression of two genes at the top of the LV is correlated that would invalidate our test).
We have a class implemented in Python that computes this, as shown below.

# Association between an LV and a trait

You can use our class `gls.GLSPhenoplier` to compute an association between an LV and a trait, in a similar way we did it in the previous cells by correlating LV603 gene weight's and neutrophil count gene associations (transformed to z-scores). However, `gls.GLSPhenoplier` takes into account correlations between the predicted expression of genes by using a generalized least squares (GLS) model.

In [43]:
from gls import GLSPhenoplier

We need to save our trait gene association to a pickle file before moving on. Here I generate a temporary file path and save:

In [45]:
with tempfile.NamedTemporaryFile(suffix=".pkl") as f:
    output_filepath = f.name
    display(output_filepath)

traits_zscores.to_pickle(output_filepath)

'/tmp/tmpx1w40_jw.pkl'

Now select an LV id and the trait id, as shown below:

In [46]:
lv_code = "LV603"

phenotype_code = traits_zscores.columns[0]
display(phenotype_code)

'Astle_et_al_2016_Neutrophil_count'

Run the model:

In [47]:
gls_model = GLSPhenoplier(
    smultixcan_result_set_filepath=output_filepath,
)

gls_model.fit_named(lv_code, phenotype_code)

It will take a few seconds. You can show the entire table of results from the [statsmodels's GLS model](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.GLS.html):

In [48]:
res = gls_model.results

In [49]:
print(res.summary())

                            GLS Regression Results                            
Dep. Variable:              phenotype   R-squared:                       0.005
Model:                            GLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     30.34
Date:                Fri, 08 Oct 2021   Prob (F-statistic):           3.76e-08
Time:                        15:58:08   Log-Likelihood:                -9329.5
No. Observations:                6450   AIC:                         1.866e+04
Df Residuals:                    6448   BIC:                         1.868e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
i              0.0059      0.013      0.474      0.6

This p-value (`3.76e-08`) is a two-sided test on the LV coefficient (`0.0698`).
You can see that the p-value is slightly less significant than the p-value from the Pearson correlation that we computed before.

This is how you can access the model's estimated parameters.

These are the coefficients:

In [50]:
res.params

i     0.005930
lv    0.069771
dtype: float64

The one-sided p-values (the ones we used in our manuscript, since we are only interested in the top genes of an LV):

In [52]:
res.pvalues_onesided

i     3.177588e-01
lv    1.882026e-08
dtype: float64

And the two-sided p-values (in case you need them or are interested in a different hypothesis):

In [53]:
res.pvalues

i     6.355177e-01
lv    3.764051e-08
dtype: float64

# Conclusions

Hopefully, now have a more clear idea of the main data matrixes involved in PhenoPLIER (matrix Z, PhenomeXcan gene-trait associations, etc).
We also see how to compute a p-value between an LV (group of genes or gene module) and a trait of interest.
To do this with your own data, you only need to compute the S-MultiXcan TWAS results (gene-based) from your GWAS summary stats.

In the next notebook (`02-LV_cell_types-...`), we'll see how to check in which tissues or cell types are our LV603'genes expressed.